In [1]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve, precision_recall_curve 

Using TensorFlow backend.


In [2]:
model_clasifyy = load_model('ff_model_clasify_cut.h5')
model_clasifyy_c = load_model('ff_model_clasify_c_cut.h5')
model_clasifyy_t = load_model('ff_model_clasify_t_cut.h5')
#doublet NN, for spatial info (model_clasifyy), cluster info(model_clasifyy_c), and combined approach(model_clasifyy_t) respectively

model_clasify = load_model('fff_model_clasify_cut-11.h5')
model_clasify_c = load_model('fff_model_clasify_c_cut-11.h5')
model_clasify_t = load_model('fff_model_clasify_t_cut-11.h5')
#triplet NN, for spatial info (model_clasify), cluster info(model_clasify_c), and combined approach(model_clasify_t) respectively

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
#test(shape = (12,)), test_c(shape = (3,)), test_t(shape = (15,)) are sample arrays for spatial, cluster, and combined.
#first 6 entries of test are from layer2 and layer4; second 6 entries of test are from layer4 and layer6.
#test_c has entries in the order of layer2, layer4, and layer6.
#test_t is the concatenation of test and test_c, with the first 12 entries of spatial info and second 3 entries of cluster info, in the above orders.

#three thresholds in prediction scores that will give around 90% efficiencies in doublet NNs.
the_cut, the_cut_c, the_cut_t = 0.95982575, 0.5185702, 0.99062485

def nn_score(test):
    doublet_score = model_clasifyy.predict((test[0:6].reshape((1, 6))))
    if doublet_score < the_cut:
        return 0
    else:
        score = model_clasify.predict(test.reshape((1, 12)))[0][0]
        return score
    
def nn_score_c(test_c):
    doublet_score = model_clasifyy_c.predict((test_c[0:2].reshape((1, 2))))
    if doublet_score < the_cut_c:
        return 0
    else:
        score = model_clasify_c.predict(test_c.reshape((1, 3)))[0][0]
        return score
    
def nn_score_t(test_t):
    sub_test_t = np.array([test_t[0], test_t[1], test_t[2], test_t[3], test_t[4], test_t[5], test_t[12], test_t[13]])
    doublet_score = model_clasifyy_t.predict((sub_test_t.reshape((1, 8))))
    if doublet_score < the_cut_t:
        return 0
    else:
        score = model_clasify_t.predict(test_t.reshape((1, 15)))[0][0]
        return score
#return 0 if it doesn't pass the doublet threshold; otherwise give the prediction score of the corresponding triplet NN.

In [4]:
##here is how to construct the triplets, both for signal and for background.
##use hits from event_id=1000 for example, all hits here have eta > 1.2

In [ ]:
#for signal and its score:

In [5]:
##dff is a dataframe used to generate signals; a particle_id (belonging to a single event_id) has at least one hit on layer_id 2, 4, and 6; 
##it's constructed by posing such selection rule to the original dataframe;

dff = np.load('ideal_final_data_cut.pkl')
ideal_particle_ids = dff['particle_id'].unique()
dff

,event_id,hit_id,eta,layer_id,module_id,nch0,nch1,particle_id,tpx,tpy,tpz,volume_id,x,y,z
20042,1000,20043.0,-0.372461,2.0,97.0,3.0,2.0,1.080958e+17,-2.771100,-0.023715,-1.515780,8.0,-33.674000,-0.446628,-12.835400
21826,1000,21827.0,-0.364603,2.0,112.0,1.0,2.0,1.080958e+17,-2.771340,-0.024289,-1.515410,8.0,-32.010101,-0.440721,-11.932400
29668,1000,29669.0,-0.455122,4.0,193.0,3.0,2.0,1.080958e+17,-2.770970,0.000340,-1.514780,8.0,-73.471901,-0.615863,-34.606300
30703,1000,30704.0,-0.453312,4.0,224.0,1.0,3.0,1.080958e+17,-2.770980,-0.001129,-1.515090,8.0,-71.580399,-0.604620,-33.572201
37345,1000,37346.0,-0.480718,6.0,313.0,2.0,2.0,1.080958e+17,-2.768900,0.027852,-1.513700,8.0,-117.384003,-0.393283,-58.627499
38162,1000,38163.0,-0.479987,6.0,364.0,2.0,2.0,1.080958e+17,-2.768750,0.026067,-1.514100,8.0,-115.445999,-0.404245,-57.565300
43651,1000,43652.0,-0.494011,8.0,391.0,2.0,2.0,1.080958e+17,-2.768500,0.058717,-1.512330,8.0,-172.957001,0.472070,-88.960899
44183,1000,44184.0,-0.493657,8.0,468.0,2.0,3.0,1.080958e+17,-2.768770,0.057969,-1.511960,8.0,-170.962006,0.427640,-87.866699
20043,1000,20044.0,-1.095952,2.0,97.0,2.0,5.0,5.944780e+17,-0.319480,0.002672,-0.442060,8.0,-34.331402,1.433860,-45.662899
21914,1000,21915.0,-1.093295,2.0,112.0,1.0,5.0,5.944780e+17,-0.319730,0.004456,-0.442086,8.0,-31.905899,1.406340,-42.300098


In [6]:
##function used to generate triplets of bothe cluster info (ideal_ch1), and xyz spatial info (ideal_x, ideal_y...) 
##locking a particular particle_id belonging to a single event_id and making the combinatorics among its hits to generate triplets;
##particle_id has to be associated with a single event_id since identical particle_ids belonging to different event_ids are different particles;
def ideal_data(par_id):
    par_info = dff.loc[dff['particle_id']==par_id]
    ideal_ch1 = []
    ideal_x = []
    ideal_y = []
    ideal_z = []
    for s in np.unique(par_info['event_id']):
        particle_info = par_info.loc[par_info['event_id'] == s]   
        par_lay2, par_lay4, par_lay6 = particle_info.loc[particle_info['layer_id'] == 2], particle_info.loc[particle_info['layer_id'] == 4], particle_info.loc[particle_info['layer_id'] == 6]
        for i in range(len(par_lay2)):
            for j in range(len(par_lay4)):
                for k in range(len(par_lay6)):
                    data_lay2, data_lay4, data_lay6 = par_lay2.iloc[i,:], par_lay4.iloc[j,:], par_lay6.iloc[k, :] 
                    x_lay2, x_lay4, x_lay6 = data_lay2['x'], data_lay4['x'], data_lay6['x']
                    y_lay2, y_lay4, y_lay6 = data_lay2['y'], data_lay4['y'], data_lay6['y']
                    z_lay2, z_lay4, z_lay6 = data_lay2['z'], data_lay4['z'], data_lay6['z']
                    ch_lay2, ch_lay4, ch_lay6 = data_lay2['nch1'], data_lay4['nch1'], data_lay6['nch1']

                    ch1_1 = [ch_lay2, ch_lay4, ch_lay6]
                    x_1 = [x_lay2, x_lay4, x_lay6]
                    y_1 = [y_lay2, y_lay4, y_lay6]
                    z_1 = [z_lay2, z_lay4, z_lay6]
                    ideal_ch1.append(ch1_1)
                    ideal_x.append(x_1)
                    ideal_y.append(y_1)
                    ideal_z.append(z_1)
    return ideal_ch1, ideal_x, ideal_y, ideal_z



In [7]:
##an example using the first particle_id;
ideal_ch1, ideal_x, ideal_y, ideal_z = ideal_data(ideal_particle_ids[0])

In [8]:
ideal_ch1

[[2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0],
 [2.0, 3.0, 2.0],
 [2.0, 3.0, 2.0],
 [2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0],
 [2.0, 3.0, 2.0],
 [2.0, 3.0, 2.0]]

In [9]:
dff.loc[dff['particle_id'] == ideal_particle_ids[0]]

,event_id,hit_id,eta,layer_id,module_id,nch0,nch1,particle_id,tpx,tpy,tpz,volume_id,x,y,z
20042,1000,20043.0,-0.372461,2.0,97.0,3.0,2.0,1.080958e+17,-2.77110,-0.023715,-1.51578,8.0,-33.674000,-0.446628,-12.835400
21826,1000,21827.0,-0.364603,2.0,112.0,1.0,2.0,1.080958e+17,-2.77134,-0.024289,-1.51541,8.0,-32.010101,-0.440721,-11.932400
29668,1000,29669.0,-0.455122,4.0,193.0,3.0,2.0,1.080958e+17,-2.77097,0.000340,-1.51478,8.0,-73.471901,-0.615863,-34.606300
30703,1000,30704.0,-0.453312,4.0,224.0,1.0,3.0,1.080958e+17,-2.77098,-0.001129,-1.51509,8.0,-71.580399,-0.604620,-33.572201
37345,1000,37346.0,-0.480718,6.0,313.0,2.0,2.0,1.080958e+17,-2.76890,0.027852,-1.51370,8.0,-117.384003,-0.393283,-58.627499
38162,1000,38163.0,-0.479987,6.0,364.0,2.0,2.0,1.080958e+17,-2.76875,0.026067,-1.51410,8.0,-115.445999,-0.404245,-57.565300
43651,1000,43652.0,-0.494011,8.0,391.0,2.0,2.0,1.080958e+17,-2.76850,0.058717,-1.51233,8.0,-172.957001,0.472070,-88.960899
44183,1000,44184.0,-0.493657,8.0,468.0,2.0,3.0,1.080958e+17,-2.76877,0.057969,-1.51196,8.0,-170.962006,0.427640,-87.866699


In [10]:
##taking two hit-arrays, with each consisting of xyz on a single layer_id, to generate spatial info
def difference_parameters(hit1, hit2):
    x1, y1, z1 = hit1[0], hit1[1], hit1[2]
    x2, y2, z2 = hit2[0], hit2[1], hit2[2]
    r1, r2 = np.sqrt(x1**2 + y1**2), np.sqrt(x2**2 + y2**2)
    phi1, phi2 = np.arctan2(y1, x1), np.arctan2(y2, x2)
    dphi = phi2 - phi1
    if dphi > np.pi:
        dphi -= 2*np.pi
    elif dphi < -np.pi:
        dphi += 2*np.pi
    dz = z2 - z1
    dr = r2 - r1
    
    phi_slope = dphi/dr
    z0 = z1 - r1 * dz/dr
    
    R1, R2 = np.sqrt(r1**2 + z1**2), np.sqrt(r2**2 + z2**2)
    theta1, theta2 = np.arccos(z1/R1), np.arccos(z2/R2)    
    eta1, eta2 = -np.log(np.tan(theta1/2.0)), -np.log(np.tan(theta2/2.0))
    deta = eta2 - eta1
    return [dphi, dz, dr, phi_slope, z0, deta]



In [11]:
##following the above example using the first particle_id
ideal_ch1, ideal_x, ideal_y, ideal_z = np.array(ideal_ch1), np.array(ideal_x), np.array(ideal_y), np.array(ideal_z)

In [12]:
# contains the x, y, z coordinates of a hit on layer_2, layer_4, layer_6
ideals_layer2 = np.array([list(ideal_x[:,0]), list(ideal_y[:,0]), list(ideal_z[:,0])]).transpose()
ideals_layer4 = np.array([list(ideal_x[:,1]), list(ideal_y[:,1]), list(ideal_z[:,1])]).transpose()
ideals_layer6 = np.array([list(ideal_x[:,2]), list(ideal_y[:,2]), list(ideal_z[:,2])]).transpose()

ideal_difference_layer24 = []
for i, j in zip(ideals_layer2, ideals_layer4):
    ideal_24 = difference_parameters(i, j)
    ideal_difference_layer24.append(ideal_24)
ideal_difference_layer24 = np.array(ideal_difference_layer24)

ideal_difference_layer46 = []
for i, j in zip(ideals_layer4, ideals_layer6):
    ideal_46 = difference_parameters(i, j)
    ideal_difference_layer46.append(ideal_46)
ideal_difference_layer46 = np.array(ideal_difference_layer46)

ideal_difference_layer24

array([[-4.88041473e-03, -2.17709007e+01,  3.97975205e+01,
        -1.22631125e-04,  5.58730049e+00, -8.26607302e-02],
       [-4.88041473e-03, -2.17709007e+01,  3.97975205e+01,
        -1.22631125e-04,  5.58730049e+00, -8.26607302e-02],
       [-4.81598709e-03, -2.07368011e+01,  3.79059905e+01,
        -1.27050818e-04,  5.58787451e+00, -8.08503310e-02],
       [-4.81598709e-03, -2.07368011e+01,  3.79059905e+01,
        -1.27050818e-04,  5.58787451e+00, -8.08503310e-02],
       [-5.38521836e-03, -2.26739006e+01,  4.14613469e+01,
        -1.29885273e-04,  5.57457236e+00, -9.05194269e-02],
       [-5.38521836e-03, -2.26739006e+01,  4.14613469e+01,
        -1.29885273e-04,  5.57457236e+00, -9.05194269e-02],
       [-5.32079073e-03, -2.16398010e+01,  3.95698169e+01,
        -1.34465892e-04,  5.57482997e+00, -8.87090277e-02],
       [-5.32079073e-03, -2.16398010e+01,  3.95698169e+01,
        -1.34465892e-04,  5.57482997e+00, -8.87090277e-02]])

In [13]:
ideal_difference = np.c_[ideal_difference_layer24, ideal_difference_layer46]
ideal_total = np.c_[ideal_difference, ideal_ch1]
ideal_total

array([[-4.88041473e-03, -2.17709007e+01,  3.97975205e+01,
        -1.22631125e-04,  5.58730049e+00, -8.26607302e-02,
        -5.03171256e-03, -2.40211983e+01,  4.39101794e+01,
        -1.14591027e-04,  5.58814030e+00, -2.55957905e-02,
         2.00000000e+00,  2.00000000e+00,  2.00000000e+00],
       [-4.88041473e-03, -2.17709007e+01,  3.97975205e+01,
        -1.22631125e-04,  5.58730049e+00, -8.26607302e-02,
        -4.88051758e-03, -2.29589996e+01,  4.19722248e+01,
        -1.16279697e-04,  5.58457859e+00, -2.48649820e-02,
         2.00000000e+00,  2.00000000e+00,  2.00000000e+00],
       [-4.81598709e-03, -2.07368011e+01,  3.79059905e+01,
        -1.27050818e-04,  5.58787451e+00, -8.08503310e-02,
        -5.09614020e-03, -2.50552979e+01,  4.58017095e+01,
        -1.11265284e-04,  5.58642901e+00, -2.74061897e-02,
         2.00000000e+00,  3.00000000e+00,  2.00000000e+00],
       [-4.81598709e-03, -2.07368011e+01,  3.79059905e+01,
        -1.27050818e-04,  5.58787451e+00, -8.08503310

In [14]:
#compute scores for a sample signal

In [15]:
spatial_score = np.array([nn_score(i) for i in ideal_difference])
spatial_score

array([0.99989784, 0.99987173, 0.9999125 , 0.99988484, 0.9999076 ,
       0.9998839 , 0.99991703, 0.9998958 ], dtype=float32)

In [16]:
cluster_score = np.array([nn_score_c(i) for i in ideal_ch1])
cluster_score

array([0.69648886, 0.69648886, 0.        , 0.        , 0.69648886,
       0.69648886, 0.        , 0.        ])

In [17]:
total_score = np.array([nn_score_t(i) for i in ideal_total])
total_score

array([0.9996264 , 0.9995372 , 0.9992022 , 0.99919564, 0.9996158 ,
       0.99952424, 0.9994295 , 0.9994611 ], dtype=float32)

In [ ]:
#for background and its score:

In [18]:
#df is a dataframe used to generate backgrounds; it's constructed by directly posing the eta > 1.2 selection to the orginal dataframe;
df = pd.read_pickle('final_data1_cut.pkl')
particle_ids = df['particle_id'].unique()
df

,event_id,hit_id,eta,layer_id,module_id,nch0,nch1,particle_id,tpx,tpy,tpz,volume_id,x,y,z
20042,1000,20043.0,-0.372461,2.0,97.0,3.0,2.0,1.080958e+17,-2.771100,-0.023715,-1.515780,8.0,-33.674000,-0.446628,-12.835400
20043,1000,20044.0,-1.095952,2.0,97.0,2.0,5.0,5.944780e+17,-0.319480,0.002672,-0.442060,8.0,-34.331402,1.433860,-45.662899
20044,1000,20045.0,-0.444202,2.0,97.0,2.0,3.0,3.242789e+17,-0.173558,-0.040211,-0.141946,8.0,-30.644300,-9.111690,-14.672900
20045,1000,20046.0,-0.375021,2.0,97.0,1.0,3.0,9.052242e+17,-0.562073,-0.241740,-0.400883,8.0,-29.223301,-13.176100,-12.305600
20046,1000,20047.0,0.000687,2.0,97.0,1.0,3.0,8.602094e+17,-0.394165,-0.084308,0.248263,8.0,-29.652300,-11.948900,0.021978
20047,1000,20048.0,-0.885688,2.0,97.0,1.0,2.0,4.954915e+16,-0.275592,-0.103508,-0.097942,8.0,-30.262899,-10.202600,-32.131599
20048,1000,20049.0,-0.305829,2.0,97.0,2.0,4.0,2.431984e+17,-1.303090,0.011327,1.367610,8.0,-33.850399,0.057954,-10.514600
20049,1000,20050.0,-0.545445,2.0,97.0,1.0,2.0,1.981644e+17,-0.299834,-0.111883,-0.105318,8.0,-29.602800,-12.090500,-18.319300
20050,1000,20051.0,-1.124728,2.0,97.0,1.0,2.0,6.170096e+17,-0.567671,-0.208399,0.294815,8.0,-30.130899,-10.580200,-43.984001
20052,1000,20053.0,-0.938916,2.0,97.0,2.0,3.0,6.170031e+17,-0.419398,-0.172060,0.353144,8.0,-29.797001,-11.535300,-34.606400


In [19]:
df_lay2, df_lay4, df_lay6 = df.loc[df['layer_id'] == 2], df.loc[df['layer_id'] == 4], df.loc[df['layer_id'] == 6]

In [20]:

# making random combinatorics of hits to generate background triplets
#10 triplets for demonstration
# same notation as in signal
all_ch1 = []
all_x = []
all_y = []
all_z = []
for l in range(10):
    i = np.random.randint(len(df_lay2))
    j = np.random.randint(len(df_lay4))
    k = np.random.randint(len(df_lay6))
    data_lay2, data_lay4, data_lay6 = df_lay2.iloc[i,:], df_lay4.iloc[j,:], df_lay6.iloc[k, :] 
    
        
    x_lay2, x_lay4, x_lay6 = data_lay2['x'], data_lay4['x'], data_lay6['x']
    y_lay2, y_lay4, y_lay6 = data_lay2['y'], data_lay4['y'], data_lay6['y']
    z_lay2, z_lay4, z_lay6 = data_lay2['z'], data_lay4['z'], data_lay6['z']
    ch_lay2, ch_lay4, ch_lay6 = data_lay2['nch1'], data_lay4['nch1'], data_lay6['nch1']
    
    ch1 = [ch_lay2, ch_lay4, ch_lay6]
    all_ch1.append(ch1)
    x = [x_lay2, x_lay4, x_lay6]
    all_x.append(x)
    y = [y_lay2, y_lay4, y_lay6]
    all_y.append(y)
    z = [z_lay2, z_lay4, z_lay6]
    all_z.append(z)


In [21]:
all_ch1, all_x, all_y, all_z = np.array(all_ch1), np.array(all_x), np.array(all_y), np.array(all_z)
all_ch1

array([[ 3.,  3.,  3.],
       [ 3.,  2.,  6.],
       [ 6.,  3.,  3.],
       [ 2.,  1.,  1.],
       [ 2.,  3.,  2.],
       [ 6.,  3.,  4.],
       [ 3.,  1.,  2.],
       [ 8.,  4.,  5.],
       [11.,  2.,  3.],
       [ 2.,  2.,  3.]])

In [22]:
# contains the x, y, z coordinates of a hit on layer_2, layer_4, layer_6
hits_layer2 = np.array([list(all_x[:,0]), list(all_y[:,0]), list(all_z[:,0])]).transpose()
hits_layer4 = np.array([list(all_x[:,1]), list(all_y[:,1]), list(all_z[:,1])]).transpose()
hits_layer6 = np.array([list(all_x[:,2]), list(all_y[:,2]), list(all_z[:,2])]).transpose()

all_difference_layer24 = []
for i, j in zip(hits_layer2, hits_layer4):
    diff_24 = difference_parameters(i, j)
    all_difference_layer24.append(diff_24)
all_difference_layer24 = np.array(all_difference_layer24)
len(all_difference_layer24)

all_difference_layer46 = []
for i, j in zip(hits_layer4, hits_layer6):
    diff_46 = difference_parameters(i, j)
    all_difference_layer46.append(diff_46)
all_difference_layer46 = np.array(all_difference_layer46)
all_difference_layer46

array([[-1.20297762e-01, -2.42776199e+02,  4.56331289e+01,
        -2.63619358e-03,  4.59713392e+02, -2.07910532e+00],
       [-3.49975745e-01,  1.51272837e+02,  4.32180398e+01,
        -8.09790881e-03, -2.55337671e+02,  1.07516417e+00],
       [-4.01356829e-01, -1.39780003e+02,  4.34562235e+01,
        -9.23588838e-03,  2.68822632e+02, -1.29166644e+00],
       [-1.93161689e+00,  1.09543697e+02,  4.21963782e+01,
        -4.57768408e-02, -2.53228050e+02,  1.17653497e+00],
       [-5.49437707e-01, -3.94434681e+01,  4.42083807e+01,
        -1.24283608e-02,  1.05316242e+02, -5.36582065e-01],
       [-1.71814284e+00, -7.71522393e+01,  4.42734908e+01,
        -3.88074853e-02,  1.32722187e+02, -6.75844036e-01],
       [-1.77045600e+00,  2.53236990e+01,  4.38519307e+01,
        -4.03735017e-02, -7.94924369e+01,  3.91721212e-01],
       [ 1.59295770e+00,  4.49790039e+01,  4.19011042e+01,
         3.80170817e-02,  2.71953972e+01, -7.94179946e-02],
       [ 1.39537252e+00,  1.18962101e+02,  4.389

In [23]:
all_difference = np.c_[all_difference_layer24, all_difference_layer46]
all_total = np.c_[all_difference, all_ch1]
all_total

array([[-2.15898791e-01,  1.03523802e+02,  3.76676028e+01,
        -5.73168386e-03, -1.19724542e+02,  1.63702880e+00,
        -1.20297762e-01, -2.42776199e+02,  4.56331289e+01,
        -2.63619358e-03,  4.59713392e+02, -2.07910532e+00,
         3.00000000e+00,  3.00000000e+00,  3.00000000e+00],
       [-5.09184016e-01,  4.13637632e+01,  4.11402552e+01,
        -1.23767831e-02, -7.30279704e+01,  1.07307346e+00,
        -3.49975745e-01,  1.51272837e+02,  4.32180398e+01,
        -8.09790881e-03, -2.55337671e+02,  1.07516417e+00,
         3.00000000e+00,  2.00000000e+00,  6.00000000e+00],
       [-1.10143319e+00,  6.90218010e+01,  3.98396363e+01,
        -2.76466678e-02, -8.82845563e+01,  1.37013783e+00,
        -4.01356829e-01, -1.39780003e+02,  4.34562235e+01,
        -9.23588838e-03,  2.68822632e+02, -1.29166644e+00,
         6.00000000e+00,  3.00000000e+00,  3.00000000e+00],
       [ 1.94565239e+00, -8.33123989e+01,  4.02825616e+01,
         4.83001159e-02,  8.72639383e+01, -1.35645585

In [24]:
all_spatial_score = np.array([nn_score(i) for i in all_difference])
all_spatial_score

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
all_cluster_score = np.array([nn_score_c(i) for i in all_ch1])
all_cluster_score

array([0.78849143, 0.        , 0.        , 0.58065057, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.32607853])

In [26]:
all_total_score = np.array([nn_score_t(i) for i in all_total])
all_total_score

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])